# Understanding the Core Data for Banking Business Rules

- Transaction, customer, and account tables record real-world operations.
- Each row in a transaction table is a transfer of value with metadata like time, type, and amount.
- Beginners sometimes mix up debit and credit, or forget to handle negative values.
- Columns like customer_id link records between datasets.
- Financial rules must respect the data structure to avoid errors.
- Always check your data for missing or illogical values before building any rules.

In [8]:
# Creating synthetic transaction data

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")


### Building synthetic transaction data

np.random.seed(42)
n_transactions = 1000
n_customers = 200
df = pd.DataFrame({
'transaction_id': range(1, n_transactions + 1),
'customer_id': np.random.choice([f'CUST_{i:04d}' for i in range(1, n_customers + 1)], n_transactions),
'amount': np.round(np.random.normal(150, 60, n_transactions), 2),
'transaction_type': np.random.choice(['Debit', 'Credit'], n_transactions),
'channel': np.random.choice(['ATM', 'Online', 'Branch', 'POS'], n_transactions),
'date': pd.date_range(start='2026-01-01', periods=n_transactions, freq='h')
})
print(df.shape)
print(df.head())

(1000, 6)
   transaction_id customer_id  amount transaction_type channel  \
0               1   CUST_0103  238.77           Credit     ATM   
1               2   CUST_0180  269.17           Credit     POS   
2               3   CUST_0093   58.62           Credit  Online   
3               4   CUST_0015   81.85            Debit  Branch   
4               5   CUST_0107  163.56           Credit  Online   

                 date  
0 2026-01-01 00:00:00  
1 2026-01-01 01:00:00  
2 2026-01-01 02:00:00  
3 2026-01-01 03:00:00  
4 2026-01-01 04:00:00  


In [9]:
#  Building a customer data table
customer_ids = [f'CUST_{i:04d}' for i in range(1, 201)]
customers = pd.DataFrame({
    'customer_id': customer_ids,
    'segment': ['Retail'] * 150 + ['Business'] * 50,
    'region': ['Metro'] * 100 + ['Regional'] * 100
})
print(customers.shape)
print(customers.head(3))

(200, 3)
  customer_id segment region
0   CUST_0001  Retail  Metro
1   CUST_0002  Retail  Metro
2   CUST_0003  Retail  Metro


In [10]:
# Creating an accounts table
accounts = pd.DataFrame({
    'account_id': [f'ACC_{i:05d}' for i in range(1, 201)],
    'customer_id': customer_ids,
    'account_type': np.random.choice(['Savings', 'Cheque', 'Credit'], size=200),
    'open_date': pd.date_range(start='2015-01-01', periods=200, freq='30D')
})
print(accounts.shape)
print(accounts.head(3))

(200, 4)
  account_id customer_id account_type  open_date
0  ACC_00001   CUST_0001       Credit 2015-01-01
1  ACC_00002   CUST_0002      Savings 2015-01-31
2  ACC_00003   CUST_0003      Savings 2015-03-02


# Beginner Example 1: A Simple Minimum Amount Rule

- Many banks block or flag transactions below a set minimum (e.g. for AML or fee reasons).
- Let us design a reusable function for this, instead of repeating your logic everywhere.

In [11]:
def is_minimum_amount(amount, threshold=50.0):
    return amount >= threshold

# Apply the rule to transaction amounts
df['meets_minimum'] = df['amount'].apply(is_minimum_amount)
print(df[['amount', 'meets_minimum']].head(6))

   amount  meets_minimum
0  238.77           True
1  269.17           True
2   58.62           True
3   81.85           True
4  163.56           True
5  200.38           True


# Beginner Example 2: Customer-Based Rule with Function

- Some banks set special rules for business customers.
- Let us make a function that checks if a customer is in the Business segment.

In [12]:
def is_business_customer(customer_id):
    segment = customers.loc[customers['customer_id'] == customer_id, 'segment'].values
    return segment[0] == 'Business' if len(segment) > 0 else False

# Test on first few customers
for cid in df['customer_id'].unique()[:6]:
    print(cid, ':', is_business_customer(cid))

CUST_0103 : False
CUST_0180 : True
CUST_0093 : False
CUST_0015 : False
CUST_0107 : False
CUST_0072 : False


# Beginner Example 3: Channel Restriction with a Function

- Sometimes a bank wants to block high-value transactions via POS terminals.
- Use a function to flag such cases for review.

In [13]:
def is_high_pos(amount, channel, limit=500):
    return channel == 'POS' and amount > limit

# Flag in dataframe
df['blocked_pos'] = df.apply(lambda x: is_high_pos(x['amount'], x['channel']), axis=1)
print(df[['amount', 'channel', 'blocked_pos']].head(8))

   amount channel  blocked_pos
0  238.77     ATM        False
1  269.17     POS        False
2   58.62  Online        False
3   81.85  Branch        False
4  163.56  Online        False
5  200.38  Online        False
6  149.33  Branch        False
7   51.72     POS        False


# Intermediate Example 1: Function for Transaction Daily Limits

- Most banking systems set daily transaction count or value limits.
- Let us write a function to count transactions for each customer per day.

In [14]:

df['date_only'] = df['date'].dt.date
def num_daily_txns(customer_id, date_only):
    return len(df[(df['customer_id'] == customer_id) & (df['date_only'] == date_only)])
count = num_daily_txns(df['customer_id'].iloc[0], df['date_only'].iloc[0])
print('Customer', df['customer_id'].iloc[0], 'had', count, 'transactions on', df['date_only'].iloc[0])

Customer CUST_0103 had 2 transactions on 2026-01-01


# Intermediate Example 2: Function to Enforce Weekly Debit Limit

- Weekly debit caps are a core anti-fraud control.
- We will create a function for total debits this week for a customer.

In [16]:

def weekly_debit_total(customer_id, this_date):
    end_date = pd.to_datetime(this_date)
    start_date = end_date - pd.Timedelta(days=6)
    mask = (df['customer_id'] == customer_id) & (df['transaction_type'] == 'Debit') & (df['date'] >= start_date) & (df['date'] <= end_date)
    return df.loc[mask, 'amount'].sum()
example_cust = df['customer_id'].iloc[5]
example_date = df['date'].iloc[5]
print('Total debit for customer', example_cust, 'from', example_date - pd.Timedelta(days=6), 'to', example_date, ':', weekly_debit_total(example_cust, example_date))

Total debit for customer CUST_0072 from 2025-12-26 05:00:00 to 2026-01-01 05:00:00 : 200.38
